In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
from sklearn import metrics # for the evaluation
from settings import CORR_GROUP, AD_THRESHOLD
from keras.callbacks import EarlyStopping
import tensorflow as tf
import logging

2022-09-28 15:21:36.131761: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 15:21:36.131834: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)


def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

In [3]:
from settings import INPUT_FILE

df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)
df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()
scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)


In [4]:
scaled_df.head()

,AE_L1,AE_L2,AE_L3,AE_SUM,C_phi_L1,C_phi_L2,C_phi_L3,F,H_TDH_I_L1_N,H_TDH_I_L2_N,...,S_L1,S_L2,S_L3,S_SUM,U_L1_L2,U_L1_N,U_L2_L3,U_L2_N,U_L3_L1,U_L3_N
ts,,,,,,,,,,,,,,,,,,,,,
2022-04-14 09:45:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.985689,0.591371,0.878465,0.769139,...,0.0,0.0,0.004865,0.001578,0.600407,0.705059,0.658637,0.614509,0.728167,0.672975
2022-04-14 09:46:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.836782,0.632546,0.878465,0.769139,...,0.0,0.0,0.004061,0.001318,0.601648,0.685304,0.636376,0.610935,0.707506,0.654408
2022-04-14 09:47:00,0.0,0.0,4.502219e-07,0.0,0.0,0.0,0.820093,0.603675,0.878465,0.769139,...,0.0,0.0,0.003887,0.001261,0.606283,0.693359,0.635031,0.610293,0.716453,0.659068
2022-04-14 09:48:00,0.0,0.0,4.911512e-07,0.0,0.0,0.0,0.739987,0.597113,0.878465,0.769139,...,0.0,0.0,0.003586,0.001163,0.628999,0.706141,0.650733,0.629983,0.731549,0.679870
2022-04-14 09:49:00,0.0,0.0,7.530985e-07,0.0,0.0,0.0,0.756175,0.536745,0.878465,0.769139,...,0.0,0.0,0.003666,0.001189,0.634475,0.702842,0.665284,0.641441,0.727198,0.687478


In [5]:
for k in CORR_GROUP:
    scaled_df['AD'] = " "
    scaled_df[k + ' AD Detected'] = " "
    scaled_df[k + ' Predict'] = " "
scaled_df.head()

,AE_L1,AE_L2,AE_L3,AE_SUM,C_phi_L1,C_phi_L2,C_phi_L3,F,H_TDH_I_L1_N,H_TDH_I_L2_N,...,ReacEc_L1 AD Detected,ReacEc_L1 Predict,C_phi_L3 AD Detected,C_phi_L3 Predict,ReacEc_L3 AD Detected,ReacEc_L3 Predict,RealE_SUM AD Detected,RealE_SUM Predict,H_TDH_U_L2_N AD Detected,H_TDH_U_L2_N Predict
ts,,,,,,,,,,,,,,,,,,,,,
2022-04-14 09:45:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.985689,0.591371,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:46:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.836782,0.632546,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:47:00,0.0,0.0,4.502219e-07,0.0,0.0,0.0,0.820093,0.603675,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:48:00,0.0,0.0,4.911512e-07,0.0,0.0,0.0,0.739987,0.597113,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:49:00,0.0,0.0,7.530985e-07,0.0,0.0,0.0,0.756175,0.536745,0.878465,0.769139,...,,,,,,,,,,


In [10]:
import random
from settings import SEED1
random.seed(SEED1)

anomaly_df = scaled_df.tail(int(0.1*len(scaled_df)))
control = 0
for index, row in anomaly_df.iterrows():
    if control != 0:
        if control > 20:
            anomaly_df.at[index, 'AD'] = True
            for k in CORR_GROUP:
                anomaly_df.at[index, k] /= 2
            control = 0
            continue

        anomaly_df.at[index, 'AD'] = True
        for k in CORR_GROUP:
            anomaly_df.at[index, k] = 0
        control += 1
        continue

    is_anomaly = random.random() < 0.0025
    if is_anomaly:
        anomaly_df.at[index, 'AD'] = True
        for k in CORR_GROUP:
            anomaly_df.at[index, k] /= 2
        control += 1
        
    else:
        anomaly_df.at[index, 'AD'] = False
        
scaled_df.iloc[-int(0.1*len(scaled_df)):] = anomaly_df
scaled_df.tail()

,AE_L1,AE_L2,AE_L3,AE_SUM,C_phi_L1,C_phi_L2,C_phi_L3,F,H_TDH_I_L1_N,H_TDH_I_L2_N,...,ReacEc_L1 AD Detected,ReacEc_L1 Predict,C_phi_L3 AD Detected,C_phi_L3 Predict,ReacEc_L3 AD Detected,ReacEc_L3 Predict,RealE_SUM AD Detected,RealE_SUM Predict,H_TDH_U_L2_N AD Detected,H_TDH_U_L2_N Predict
ts,,,,,,,,,,,,,,,,,,,,,
2022-05-11 20:54:00,0.999940,0.999937,0.999939,0.999939,0.964400,0.974470,0.961782,0.520997,0.534157,0.473966,...,,,,,,,,,,
2022-05-11 20:55:00,0.999971,0.999969,0.999970,0.999970,0.964736,0.978975,0.960113,0.506562,0.522807,0.462906,...,,,,,,,,,,
2022-05-11 20:56:00,0.999995,0.999993,0.999994,0.999994,0.764903,0.866177,0.971629,0.525591,0.737308,0.592497,...,,,,,,,,,,
2022-05-11 20:57:00,1.000000,1.000000,1.000000,1.000000,0.000000,0.431670,0.992824,0.498031,0.878465,0.409323,...,,,,,,,,,,
2022-05-11 20:58:00,1.000000,1.000000,1.000000,1.000000,0.000000,0.414770,0.993229,0.517435,0.878465,0.384148,...,,,,,,,,,,


In [13]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging
import pickle

models_linreg = ['ReacEc_L1', 'ReacEc_L3', 'RealE_SUM']
ignore = ['F', 'C_phi_L3']

for var in CORR_GROUP:
    if var in ignore: continue
    print(var + " started script")
    model = None
    if var in models_linreg: model = pickle.load(open(f'models/{var}_model.sav', 'rb'))
    else: model = pickle.load(open(f'models/autosklearn_{var}', 'rb'))
    features = []
    counter = 0
    history_window = 15
    for index, row in scaled_df.iloc[-int(0.1*len(scaled_df)):].iterrows():

        if counter >= history_window:
            if row['AD'] == True or row['AD'] == False:
                
                res = model.predict(np.array(features).reshape(1, -1))
                
                scaled_df.at[index, var + ' Predict'] = res
                ad_detected = abs(res - row[var]) > AD_THRESHOLD[var]
                scaled_df.at[index, var + ' AD Detected'] = ad_detected
            features = features[len(CORR_GROUP[var]):]
            
        counter += 1
        predictors = row[CORR_GROUP[var]]
        features += predictors.to_list()
    

P_SUM started script


KeyboardInterrupt: 

In [11]:
anomaly_df = scaled_df.tail(int(0.1*len(scaled_df)))
results = {k:[0,0,0,0] for k in CORR_GROUP}
for index, row in anomaly_df.iterrows():
    for k in CORR_GROUP:
        if row[k + ' AD']:
            if row[k + ' AD Detected']: results[k][0]+=1
            else: results[k][1]+=1
        else:
            if row[k + ' AD Detected']: results[k][2]+=1
            else: results[k][3]+=1
        break
print(results)

{'P_SUM': [203, 0, 3461, 291], 'U_L1_N': [0, 0, 0, 0], 'I_SUM': [0, 0, 0, 0], 'H_TDH_I_L3_N': [0, 0, 0, 0], 'F': [0, 0, 0, 0], 'ReacEc_L1': [0, 0, 0, 0], 'C_phi_L3': [0, 0, 0, 0], 'ReacEc_L3': [0, 0, 0, 0], 'RealE_SUM': [0, 0, 0, 0], 'H_TDH_U_L2_N': [0, 0, 0, 0]}


In [30]:
from tensorflow import keras
from AttentionBiLSTM import attention

for var in ['U_L1_N', 'I_SUM', 'F', 'C_phi_L3']:
    model = keras.models.load_model(f'models/{var}_autokeras.h5')
    features = []
    counter = 0
    history_window = 15
    for index, row in scaled_df.iterrows():
        if counter >= history_window:
            tensor = np.array(features).reshape((1, 1, len(features)))
            res = model.predict(tensor)
            ad_detected = abs(res - row[var]) > AD_THRESHOLD[var]
            scaled_df.at[index, var + ' AD'] = ad_detected
            features = features[len(CORR_GROUP[var]):]

        counter += 1
        predictors = row[CORR_GROUP[var]]
        features += predictors.to_list()
    

In [31]:
for i in ['P_SUM', 'U_L1_N', 'I_SUM', 'F', 'C_phi_L3']:
    df[i + ' AD'] = scaled_df[i + ' AD']

df.to_csv('data/anomaly_detection_data.csv')

In [18]:
import matplotlib.pyplot as plt
ad = pd.read_csv('results/ad_results.csv')
for k in CORR_GROUP:
    print(ad[[k, k + ' AD Prevision']].head())

      P_SUM P_SUM AD Prevision
0  0.000585     [[0.04496545]]
1  0.000577     [[0.04491363]]
2  0.000692     [[0.04485293]]
3  0.000629     [[0.04483562]]
4  0.000698     [[0.04482092]]
     U_L1_N U_L1_N AD Prevision
0  0.482762      [[0.47984925]]
1  0.475593      [[0.47476748]]
2  0.530821      [[0.46775433]]
3  0.448186       [[0.5232624]]
4  0.412154      [[0.43610483]]
      I_SUM I_SUM AD Prevision
0  0.377139      [[0.5630058]]
1  0.372632      [[0.3737879]]
2  0.445347     [[0.36898273]]
3  0.403816     [[0.43896848]]
4  0.418921     [[0.39983198]]
   H_TDH_I_L3_N H_TDH_I_L3_N AD Prevision
0      0.618515            [[0.64999306]]
1      0.620963             [[0.6222509]]
2      0.633001             [[0.6247624]]
3      0.618898            [[0.63668644]]
4      0.554420             [[0.6228265]]
          F F AD Prevision
0  0.601706  [[0.5643054]]
1  0.574147  [[0.6028594]]
2  0.562970  [[0.5744198]]
3  0.522966  [[0.5632101]]
4  0.538714  [[0.5219256]]
   ReacEc_L1 ReacEc_L1

ts
2022-05-09 03:04:00    0.000585
2022-05-09 03:05:00    0.000577
2022-05-09 03:06:00    0.000692
2022-05-09 03:07:00    0.000629
2022-05-09 03:08:00    0.000698
Name: P_SUM, dtype: float64